In [10]:
!pip install youtokentome

    100% |████████████████████████████████| 92kB 9.2MB/s eta 0:00:01
    100% |████████████████████████████████| 81kB 3.8MB/s ta 0:00:011
  Running setup.py bdist_wheel for youtokentome ... done
  Stored in directory: /root/.cache/pip/wheels/30/65/29/959c7ddc32cd220a9e633c5818802efe41a990c476220aed69
Successfully built youtokentome
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from tqdm.auto import tqdm

from nltk.tokenize import word_tokenize, wordpunct_tokenize
from tqdm.auto import tqdm

import pandas as pd

import numpy as np

from torch.utils.data import Dataset, DataLoader

In [12]:
import math
import random
import json
import torch
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

from matplotlib import pyplot as plt

import youtokentome as yttm

In [13]:
#ИЗМЕНИТЬ ПУТЬ В ЗАВИСИМОСТИ ОТ ТОГО ГДЕ ЗАПУСКАЕМ КОД
try:
    df = pd.read_csv("ctx_quest.csv")
except:
    df = pd.read_csv("/Users/lilyakhoang/input/question_generation/ctx_quest.csv")

df.head()

,Unnamed: 0,context,question
0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?
1,1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...
2,2,"Managed by her father, Mathew Knowles, the gro...",When did Beyonce leave Destiny's Child and bec...
3,3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?
4,4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?


In [14]:
df_nonan = df.dropna()
df_nonan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86610 entries, 0 to 86820
Data columns (total 3 columns):
Unnamed: 0    86610 non-null int64
context       86610 non-null object
question      86610 non-null object
dtypes: int64(1), object(2)
memory usage: 2.6+ MB


In [15]:
# # для "обучения" bpe модели нам нужно сохранить данные для обучения в отдельный файл
# # где будут построчно храниться тексты
# err = 0
# try:
#     f = open('/Users/lilyakhoang/input/for_bpe_ctx_quest.txt', 'w')
#     bpe_model_address = '/Users/lilyakhoang/input/for_bpe_ctx_quest.txt'
# except:
#     f = open('for_bpe_ctx_quest.txt', 'w')
#     bpe_model_address = 'for_bpe_ctx_quest.txt'
# for que in df_nonan.context:
#     try:
#         f.write(que + '\n')
#     except:
#         err += 1
# for que in df_nonan.question:
#     try:
#         f.write(que + '\n')
#     except:
#         err += 1
# f.close()
# print(err)

In [15]:
# обучаем
vocab_size = 16000
model_path = 'bpe.model'

# yttm.BPE.train(data='for_bpe_ctx_quest.txt', vocab_size=vocab_size, model=model_path)

In [16]:
tokenizer = yttm.BPE(model=model_path)

# Убрал BOS и EOS

In [17]:
# давайте токенизируем наш датасет
# токенизирую батчами, потому что так быстрее
# также в начало добавляем токен bos (begin of sentence)

tokenized_ctx = []
tokenized_quest = []
batch_size = 256

for i_batch in tqdm(range(math.ceil(len(df_nonan.context) / batch_size))):
    
    tokenized_ctx.extend(tokenizer.encode(list(df_nonan.context[i_batch*batch_size:(i_batch+1)*batch_size])))

for i_batch in tqdm(range(math.ceil(len(df_nonan.question) / batch_size))):

    tokenized_quest.extend(tokenizer.encode(list(df_nonan.question[i_batch*batch_size:(i_batch+1)*batch_size])))

Пример с переводом:
1. что подаем на вход энкодеру: мама мыла раму
1. что подаем на вход декодеру: bos mom was washing the frame
1. наши таргеты (от чего считать лосс): mom was washing the frame eos

То есть наши таргеты это почти тоже самое, что мы подаем на вход декодеру, но по сути это смещенная вправо последовательность. Мы обрубаем наши тексты до максимальной длины - 1 (не учитывая bos и eos теги), потому что в последовательности 2 и 3 мы добавим по одному тегу (bos и eos). 

In [19]:
encoder_sequence = 'мама мыла раму'.split()
decoder_sequence = 'bos mom was washing the frame'.split()
target_sequence = 'mom was washing the frame eos'.split()

In [20]:
len(decoder_sequence) == len(target_sequence)

True

In [21]:
for n in range(len(decoder_sequence)):
    print(f'По слову "{decoder_sequence[n]}" предсказываем слово "{target_sequence[n]}"')
    print('Или с точки зрения rnn')
    print(f'По слову "{decoder_sequence[n]}" при условии того, что в памяти есть {decoder_sequence[:n]} предсказываем слово "{target_sequence[n]}"')
    print()

По слову "bos" предсказываем слово "mom"
Или с точки зрения rnn
По слову "bos" при условии того, что в памяти есть [] предсказываем слово "mom"

По слову "mom" предсказываем слово "was"
Или с точки зрения rnn
По слову "mom" при условии того, что в памяти есть ['bos'] предсказываем слово "was"

По слову "was" предсказываем слово "washing"
Или с точки зрения rnn
По слову "was" при условии того, что в памяти есть ['bos', 'mom'] предсказываем слово "washing"

По слову "washing" предсказываем слово "the"
Или с точки зрения rnn
По слову "washing" при условии того, что в памяти есть ['bos', 'mom', 'was'] предсказываем слово "the"

По слову "the" предсказываем слово "frame"
Или с точки зрения rnn
По слову "the" при условии того, что в памяти есть ['bos', 'mom', 'was', 'washing'] предсказываем слово "frame"

По слову "frame" предсказываем слово "eos"
Или с точки зрения rnn
По слову "frame" при условии того, что в памяти есть ['bos', 'mom', 'was', 'washing', 'the'] предсказываем слово "eos"



Хорошо было бы переписать loader с такими последовательностями.

In [18]:
class WordData(torch.utils.data.Dataset):
    def __init__(self, context_list, questions_list, context_len, questions_len, pad_index, eos_index):
        self.context_list = context_list
        self.questions_list = questions_list
        
        self.context_len = context_len
        self.questions_len = questions_len
        
        self.pad_index = pad_index
        self.eos_index = eos_index
        
    def __len__(self):
        return len(self.context_list)
    
    def __getitem__(self, index):
        
        encoder_sequence = self.context_list[index][:self.context_len]
        decoder_sequence = self.questions_list[index][:self.questions_len]
        
        target_sequence = decoder_sequence[:] + [self.eos_index]
        decoder_sequence = decoder_sequence[:]
        
        encoder_pads = [self.pad_index] * (self.context_len - len(encoder_sequence))
        decoder_pads = [self.pad_index] * (self.questions_len - len(decoder_sequence))
        target_pads = [self.pad_index] * (self.questions_len - len(decoder_sequence))
        
        encoder_sequence = torch.tensor(encoder_sequence + encoder_pads).long()
        decoder_sequence = torch.tensor(decoder_sequence + decoder_pads).long()
        target_sequence = torch.tensor(target_sequence + target_pads).long()
        
        return encoder_sequence, decoder_sequence, target_sequence

In [19]:
batch_size = 64

context_len = 80
quest_len = 20

pad_index = 0
eos_index = 3

In [20]:
validation_start_index = int(len(tokenized_ctx) * 0.1)

In [21]:
train_dataset = WordData(context_list=tokenized_ctx[:-validation_start_index],
                         questions_list = tokenized_quest[:-validation_start_index],
                         context_len=context_len, questions_len = quest_len, pad_index=pad_index, eos_index=eos_index)

validation_dataset = WordData(context_list=tokenized_ctx[-validation_start_index:],
                              questions_list = tokenized_quest[-validation_start_index:],
                         context_len=context_len, questions_len = quest_len, pad_index=pad_index, eos_index=eos_index)

len(train_dataset), len(validation_dataset)

(77949, 8661)

In [22]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=64)

In [289]:
for encoder_sequence, decoder_sequence, target_sequence in train_loader:
    print(encoder_sequence[0], decoder_sequence[0], target_sequence[0])
    break

tensor([ 3297,  1394,  1304, 13539, 12243,  1838,  4502,  4113,  8059, 13895,
          100,    90,    65,   197,     9,  1444,   112,    76,  1348,     5,
           42,    73,    62,    51,  2250,  1398,    47,  4690,  2117,  2811,
         6982,  2156,  8762,  1352,  1306,  1959,  2339,  2087,  3046,  7582,
         2087,  2083,  7536,  1312,  1875, 11109,  1365,  2117,  1312,  5267,
         1305,  4274, 13461, 15846,  2402,  4829,  1305,  2734, 10861,  1312,
         1317,  9041, 12046,  1363,  1285, 12745,  1312,  7309,  1319, 13738,
         1305,  1287,  2535,  7915,  1363,  2018,  8292,  1300,  9914,  8155]) tensor([1800, 1503, 7635, 2294, 4596, 2454,   36,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]) tensor([1800, 1503, 7635, 2294, 4596, 2454,   36,    3,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0])


In [24]:
encoder_sequence.shape

torch.Size([64, 80])

In [25]:
decoder_sequence.shape

torch.Size([64, 20])

In [26]:
target_sequence.shape

torch.Size([64, 21])

In [ ]:
эмбеддинг энкодер последовательности
encoder_embed = self.embedding_layer(encoder_sequence)
# эмбеддинг декодер последовательности
decoder_embed = self.embedding_layer(decoder_sequence)

# пропускаем энкодер последовательность через энкодер, получаем хайден и память
encoder_hidden, encoder_mem = self.encoder(encoder_embed)

# добавляем последний (с точки зрения времени) хайден в начало эмбеддингов декодер последовательности
# то есть по сути первый токен (эмбеддинг токена) декодера - это последний хайден из энкодера
decoder_embed = torch.cat([encoder_hidden[:, -1, :].unsqueeze(1), decoder_embed], dim=1)

# пропускаем декодер последовательность через декодер, получаем хайден и память
# в качестве инициализирующей памяти вставляем память с энкодера
decoder_hidden, _ = self.decoder(decoder_embed, encoder_mem)

# предсказываем слова
prediction = self.head(decoder_hidden)

In [43]:
class SimpleSequence2Sequence(torch.nn.Module):
    
    def __init__(self, vocab_size=16000, embedding_size=256,
                 hidden_size=256, num_layers=2, dropout=0.3, padding_idx=0):
        
        super().__init__()
        
        self.embedding_layer = torch.nn.Embedding(vocab_size, embedding_size, padding_idx)
        
        self.encoder = torch.nn.GRU(embedding_size, hidden_size, 
                                    num_layers=num_layers, dropout=dropout, batch_first=True)
        self.attn = nn.Linear(self.hidden_size * 2, 80)
        
        self.decoder = torch.nn.GRU(hidden_size, hidden_size, 
                                    num_layers=num_layers, dropout=dropout, batch_first=True)
        
        self.head = torch.nn.Linear(hidden_size, vocab_size)
        
    def forward(self, encoder_sequence, decoder_sequence):
        
        encoder_embed = self.embedding_layer(encoder_sequence)
        decoder_embed = self.embedding_layer(decoder_sequence)
        print("encoder_embed",encoder_embed.shape)
        encoder_hidden, encoder_mem = self.encoder(encoder_embed)
        print("encoder_hidden",encoder_hidden.shape)
        decoder_embed = torch.cat([encoder_hidden[:, -1, :].unsqueeze(1), decoder_embed], dim=1)
        print("decoder_embed",decoder_embed.shape)
        decoder_hidden, _ = self.decoder(decoder_embed, encoder_mem)
        print("decoder_hidden",decoder_hidden.shape)
        prediction = self.head(decoder_hidden)
        print("prediction",prediction.shape)
        print("====")

        return prediction

In [60]:
class SimpleSequence2Sequence_attention(torch.nn.Module):
    
    def __init__(self, vocab_size=16000, embedding_size=256,
                 hidden_size=256, num_layers=2, dropout=0.3, padding_idx=0):
        
        super().__init__()
        self.hidden_size = hidden_size
        
        self.embedding_layer = torch.nn.Embedding(vocab_size, embedding_size, padding_idx)
        
        self.encoder = torch.nn.GRU(embedding_size, hidden_size, 
                                    num_layers=num_layers, dropout=dropout, batch_first=True)
        self.attn = nn.Linear(self.hidden_size, 80)
        
        self.decoder = torch.nn.GRU(hidden_size, hidden_size, 
                                    num_layers=num_layers, dropout=dropout, batch_first=True)
        
        self.head = torch.nn.Linear(hidden_size, vocab_size)
        
    def forward(self, encoder_sequence, decoder_sequence, debug = True):
        
        encoder_embed = self.embedding_layer(encoder_sequence)
        decoder_embed = self.embedding_layer(decoder_sequence)
        if debug == True: print("encoder_embed",encoder_embed.shape)
        encoder_hidden, encoder_mem = self.encoder(encoder_embed)
        if debug == True: print("encoder_hidden",encoder_hidden.shape)
        attn_weights = torch.softmax(self.attn(encoder_hidden), dim=1)
        if debug == True: print("attn_weights.shape {}, encoder_hidden.shape {}".format(attn_weights.shape, encoder_hidden.shape))
        attn_applied = torch.bmm(attn_weights,encoder_hidden)
        decoder_embed = torch.cat([attn_applied[:, -1, :].unsqueeze(1), decoder_embed], dim=1)
        if debug == True: print("decoder_embed",decoder_embed.shape)
        decoder_hidden, _ = self.decoder(decoder_embed, encoder_mem)
        if debug == True: print("decoder_hidden",decoder_hidden.shape)
        prediction = self.head(decoder_hidden)
        if debug == True: 
            print("prediction",prediction.shape)
            print("====")

        return prediction

In [ ]:
for timestamp in x.size(1)
    current_step = decoder_embed[:, timestamp, :]
    current_decoder_rnn_hidden, decoder_mem = self.decoder(current_step, decoder_mem)
    current_decoder_attention = self.attn(current_decoder_rnn_hidden, current_decoder_rnn_hidden)
    ord_prediction = self.head(torch.cat([current_decoder_attention, current_decoder_rnn_hidden]))

In [171]:

ENCODER_WORDS = 2
DECODER_WORDS = 3
EMBEDDING_DIM = 4

In [172]:
BATCH_SIZE = 32

encoder_hidden = torch.rand(BATCH_SIZE, ENCODER_WORDS, EMBEDDING_DIM)
decoder_hidden = torch.rand(BATCH_SIZE, DECODER_WORDS, EMBEDDING_DIM)
encoder_hidden.shape, decoder_hidden.shape

(torch.Size([32, 2, 4]), torch.Size([32, 3, 4]))

In [174]:
decoder_hidden.shape, encoder_hidden.transpose(1, 2).shape

(torch.Size([32, 3, 4]), torch.Size([32, 4, 2]))

In [177]:
attention_scores = torch.bmm(decoder_hidden, encoder_hidden.transpose(1, 2))
attention_scores.shape

torch.Size([32, 3, 2])

In [179]:
attention_distribution = torch.softmax(attention_scores, 2)
attention_distribution.shape, encoder_hidden.shape

(torch.Size([32, 3, 2]), torch.Size([32, 2, 4]))

In [180]:
attention_vectors = torch.bmm(attention_distribution, encoder_hidden)
attention_vectors.shape

torch.Size([32, 3, 4])

In [182]:
decoder_hidden.shape, attention_vectors.shape

(torch.Size([32, 3, 4]), torch.Size([32, 3, 4]))

In [184]:
decoder_with_attention = torch.cat([decoder_hidden, attention_vectors], dim=-1)
decoder_with_attention.shape

torch.Size([32, 3, 8])

In [ ]:
аттеншин между текущим словом в декодере и encoder_hidden

In [ ]:
decoder_mem = encoder_mem
for timestamp in x.size(1)
current_step = decoder_embed[:, timestamp, :]
current_decoder_rnn_hidden, decoder_mem = self.decoder(current_step, decoder_mem)
current_decoder_attention = self.attn(current_decoder_rnn_hidden, current_decoder_rnn_hidden)
word_prediction = self.head(torch.cat([current_decoder_attention, current_decoder_rnn_hidden]))

In [315]:
class SimpleSequence2Sequence_attention_cycle(torch.nn.Module):
    def __init__(self, vocab_size=16000, embedding_size=256,
                 hidden_size=256, num_layers=2, dropout=0.3, padding_idx=0):
        
        super().__init__()
        self.hidden_size = hidden_size
        
        self.embedding_layer = torch.nn.Embedding(vocab_size, embedding_size, padding_idx)
        
        self.encoder = torch.nn.GRU(embedding_size, hidden_size, 
                                    num_layers=num_layers, dropout=dropout, batch_first=True)
        self.attn = nn.Linear(self.hidden_size, self.hidden_size)
        
        self.decoder = torch.nn.GRU(hidden_size, hidden_size, 
                                    num_layers=num_layers, dropout=dropout, batch_first=True)
        
        self.head = torch.nn.Linear(hidden_size*2, vocab_size)
        
    def forward(self, encoder_sequence, decoder_sequence, debug = True):
        
        encoder_embed = self.embedding_layer(encoder_sequence)
        encoder_hidden, encoder_mem = self.encoder(encoder_embed)
        decoder_mem = encoder_mem#.transpose(0,1).transpose(1,2)
        decoder_embed = self.embedding_layer(decoder_sequence)
        decoder_embed = torch.cat([encoder_hidden[:, -1, :].unsqueeze(1), decoder_embed], dim=1)
        word_predictions = torch.zeros(64,21,16000)
        for timestamp in range(decoder_embed.size(1)):
            current_step = decoder_embed[:, timestamp, :]
            current_decoder_rnn_hidden, decoder_mem = self.decoder(current_step.unsqueeze(1), decoder_mem)
            current_decoder_attention = self.attn(current_decoder_rnn_hidden)
            print("current_decoder_attention", current_decoder_attention.shape)
            cat = torch.cat([current_decoder_attention, current_decoder_rnn_hidden], dim = -1)
            word_prediction = self.head(cat)
            word_predictions[:,timestamp,:] = word_prediction.squeeze(1)
        
        return word_predictions

In [404]:
class SimpleSequence2Sequence_attention_cycle(torch.nn.Module):
    def __init__(self, vocab_size=16000, embedding_size=256,
                 hidden_size=256, num_layers=2, dropout=0.3, padding_idx=0):
        
        super().__init__()
        self.hidden_size = hidden_size
        
        self.embedding_layer = torch.nn.Embedding(vocab_size, embedding_size, padding_idx)
        
        self.encoder = torch.nn.GRU(embedding_size, hidden_size, 
                                    num_layers=num_layers, dropout=dropout, batch_first=True)
        
        self.decoder = torch.nn.GRU(hidden_size, hidden_size, 
                                    num_layers=num_layers, dropout=dropout, batch_first=True)
        
        self.head = torch.nn.Linear(hidden_size*2, vocab_size)
        
    def forward(self, encoder_sequence, decoder_sequence, debug = True):
        encoder_embed = self.embedding_layer(encoder_sequence)
        encoder_hidden, encoder_mem = self.encoder(encoder_embed)
        encoder_hidden, encoder_mem = torch.nn.utils.rnn.pad_packed_sequence(encoder_hidden, batch_first=True)
        decoder_mem = encoder_mem#.transpose(0,1).transpose(1,2)
        decoder_embed = self.embedding_layer(decoder_sequence)
        decoder_embed = torch.cat([encoder_hidden[:, -1, :].unsqueeze(1), decoder_embed], dim=1)
        word_predictions = torch.zeros(64,21,16000)
        for timestamp in range(decoder_embed.size(1)):
            current_step = decoder_embed[:, timestamp, :]
            current_decoder_rnn_hidden, decoder_mem = self.decoder(current_step.unsqueeze(1), decoder_mem)
            attention_scores = torch.bmm(current_decoder_rnn_hidden, encoder_hidden.transpose(1, 2))
            attention_distribution = torch.softmax(attention_scores, 2)
            current_decoder_attention = torch.bmm(attention_distribution, encoder_hidden)
            cat = torch.cat([current_decoder_attention, current_decoder_rnn_hidden], dim = -1)
            word_prediction = self.head(cat)
            word_predictions[:,timestamp,:] = word_prediction.squeeze(1)
        
        return word_predictions

In [402]:
# current_decoder_attention torch.Size([64, 1, 256])

In [403]:
model = SimpleSequence2Sequence_attention_cycle()
model.to(device)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)
train(model, train_loader, criterion, optimizer)

tensor([[ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 7,  7,  7,  ...,  7,  7,  7],
        ...,
        [ 0,  0,  0,  ...,  0,  0,  0],
        [35, 35, 35,  ..., 35, 35, 35],
        [ 0,  0,  0,  ...,  0,  0,  0]])
tensor([[80, 80, 80,  ..., 80, 80, 80],
        [80, 80, 80,  ..., 80, 80, 80],
        [73, 73, 73,  ..., 73, 73, 73],
        ...,
        [80, 80, 80,  ..., 80, 80, 80],
        [45, 45, 45,  ..., 45, 45, 45],
        [80, 80, 80,  ..., 80, 80, 80]])


RuntimeError: 'lengths' argument should be a 1D CPU int64 tensor

In [343]:
def train(model, iterator, criterion, optimizer):
    
   # Put the model in training mode!
    model.train()

    losses_list = []
    epoch_loss = 0
    progress_bar = tqdm(iterator, total=len(iterator))
    
    for encoder_sequence, decoder_sequence, target_sequence in progress_bar:
        
        optimizer.zero_grad()
        encoder_sequence = encoder_sequence.to(device)
        decoder_sequence = decoder_sequence.to(device)
        target_sequence = target_sequence.to(device)

        output = model(encoder_sequence, decoder_sequence)
#         print("model output", output.shape)
        output = output.view(-1, output.size(-1))
        
        target_sequence = target_sequence.view(-1)
#         print("to be sent to criterion", output.shape, target_sequence.shape)
        loss = criterion(output, target_sequence)
        losses_list.append(loss.item())
        
        loss.backward()
        optimizer.step()
        
        progress_bar.set_postfix(train_loss = np.mean(losses_list[-500:]))
    return losses_list

In [ ]:
pad_idx = tokenizer.vocab().index("<PAD>")
eos_idx = tokenizer.vocab().index("<EOS>")
sos_idx = tokenizer.vocab().index("<BOS>")
# Size of embedding_dim should match the dim of pre-trained word embeddings!
embedding_dim = 300
hidden_dim = 300
vocab_size = len(tokenizer.vocab())
model = My_seq2seq_attention(embedding_dim,
                 hidden_dim, 
                 vocab_size, 
                 device, pad_idx, eos_idx, sos_idx).to(device)
optimizer = optim.Adam([param for param in model.parameters() if param.requires_grad == True], lr=1.0e-3)
criterion = nn.CrossEntropyLoss(ignore_index = pad_idx)

N_EPOCHS = 20
train_losses = []
for epoch in range(N_EPOCHS):
    train_loss = train(model, train_loader, criterion, optimizer, epoch)
#     print (train_loss)
    train_losses.append(train_loss)
    if min(train_losses) == train_loss and len(train_losses) > 1:
        torch.save(model.state_dict, "best_seq2seq_attention")
        torch.save(optimizer.state_dict, "best_Adam_state_dict_attention")
    
    torch.save(model.state_dict, "last_seq2seq_attention")
    torch.save(optimizer.state_dict, "Adam_state_dict_attention")
    
    #early stopping
    validation_losses = []
    test_loss = evaluate(model, validation_loader)
    validation_losses.append(test_loss)
    
    if len(validation_losses) > 1 and validation_losses[epoch] > validation_losses[epoch-1]:
        print("stop")
        break
#     break

In [16]:
class EncoderRNN_inside_class(nn.Module):
    def __init__(self, hidden_size, embedding_size, embedding):#input_size
        super(EncoderRNN_inside_class, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = embedding
        self.gru = nn.GRU(embedding_size, hidden_size, batch_first = True)

    def forward(self, input, hidden, debug = False):
        if debug == True: 
            print("====ENCODING_FORWARD====")
            print("input.shape", input.shape)
#         embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.embedding(input)
        output = embedded
        if debug == True: 
            print("embedded/output.shape",embedded.shape,"hidden.shape", hidden.shape  )
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

In [17]:
# исправленный
# необязательно задавать хайден, читай доку
class EncoderRNN_inside_class(nn.Module):
    def __init__(self, hidden_size, embedding_size, embedding):#input_size
        super(EncoderRNN_inside_class, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = embedding
        self.gru = nn.GRU(embedding_size, hidden_size, batch_first = True)

    def forward(self, sequence):

        embedded = self.embedding(sequence)
        output, hidden = self.gru(embedded)
        
        return output, hidden

In [18]:
class AttentionDecoder_inside_class(nn.Module):
    def __init__(self, embedding, embedding_size,
                 hidden_size, output_size):
        super(AttentionDecoder_inside_class, self).__init__()
        self.embedding = embedding
        self.hidden_size = hidden_size
        self.attn = nn.Linear(self.hidden_size * 2, 80)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, output_size)
        
    def forward(self, input, hidden, encoder_outputs, debug = False):
        embedded = self.embedding(input)
        if debug == True:
            print("embedded[0].shape {}, hidden.shape {}".format(embedded[0].shape, hidden.shape))
        attn_weights = torch.softmax(self.attn(torch.cat((embedded[0], hidden), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),encoder_outputs.unsqueeze(0))
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)
        hidden = hidden.unsqueeze(0)
        output = F.relu(output)
        if debug == True: 
            print("output.shape {}, hidden.shape {}".format(output.shape,hidden.shape ))
        output, hidden = self.gru(output, hidden)
        output = F.log_softmax(self.out(output[0]), dim=1)
        hidden = hidden.squeeze(1)
        return output, hidden

In [19]:
class My_seq2seq_attention(nn.Module):
    def __init__(self, embedding_size, hidden_size, vocab_size, 
                 device, pad_idx, eos_idx, sos_idx):
        super(My_seq2seq_attention, self).__init__()
        
        self.vocab_size = vocab_size
        
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        
        # Encoder network
        self.encoder = EncoderRNN_inside_class(hidden_size, 
                               embedding_size, 
                               self.embedding)
        
        # Decoder network        
        self.decoder = AttentionDecoder_inside_class(self.embedding,
                               embedding_size,
                              hidden_size,
                              vocab_size)
        
        self.pad_idx = pad_idx
        self.eos_idx = eos_idx
        self.sos_idx = sos_idx
        self.device = device
        
    def forward(self, input_sequence, output_sequence, debug = False):
        

        encoder_output, encoder_hidden = self.encoder(input_sequence)
        batch_size = len(input_sequence)
        outputs = torch.zeros(batch_size, 20, self.vocab_size).to(self.device)
        
        for batch_element_index in range(batch_size):
            
            target_tensor = output_sequence[batch_element_index, :]
                
            decoder_input = torch.tensor([[self.sos_idx]], device=device)

            decoder_hidden = encoder_hidden[:,batch_element_index,:]

            encoder_output_in_current_index = encoder_output[batch_element_index,:,:]

            for di in range(20):
                
                decoder_output, decoder_hidden = self.decoder(
                    decoder_input, decoder_hidden, encoder_output_in_current_index)
                
                outputs[batch_element_index,di,:] = decoder_output
                
                if target_tensor[di] != self.pad_idx:
                    decoder_input = torch.tensor([target_tensor[di]], device=device).unsqueeze(1)
                else:
                    break
        return outputs

In [20]:
def train(model, iterator, criterion, optimizer,epoch_number, debug = False):
   # Put the model in training mode!
    model.train()

    losses_list = []
    epoch_loss = 0
    progress_bar = tqdm(enumerate(iterator), total=len(iterator), desc='Epoch {}'.format(epoch_number + 1))
    for idx, batch in progress_bar:
        optimizer.zero_grad()
        input_seq = torch.tensor(batch[0]).to(device)
        target_tokens = torch.tensor(batch[1]).to(device)

        output = model(input_seq, target_tokens)

        output = output.view(-1, output.size(-1))
        target_tokens = target_tokens.view(-1)
        loss = criterion(output, target_tokens)
        losses_list.append(float(loss))
        progress_bar.set_postfix(train_loss = np.mean(losses_list[-100:]))
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
#         break
    return epoch_loss / len(iterator)

In [21]:
def evaluate (model, iterator):
    epoch_loss = 0
    progress_bar = tqdm(enumerate(iterator), total=len(iterator))
    for idx, batch in progress_bar:
        with torch.no_grad():
            input_seq = torch.tensor(batch[0]).to(device)
            target_tokens = torch.tensor(batch[1]).to(device)
            output = model(input_seq, target_tokens)
            output = output.view(-1, output.size(-1))
            target_tokens = target_tokens.view(-1)
            loss = criterion(output, target_tokens)
            epoch_loss += loss.item()
            progress_bar.set_postfix(train_loss = np.mean(losses_list[-100:]))
        
    return epoch_loss / len(iterator)

In [22]:
pad_idx = tokenizer.vocab().index("<PAD>")
eos_idx = tokenizer.vocab().index("<EOS>")
sos_idx = tokenizer.vocab().index("<BOS>")
# Size of embedding_dim should match the dim of pre-trained word embeddings!
embedding_dim = 300
hidden_dim = 300
vocab_size = len(tokenizer.vocab())
model = My_seq2seq_attention(embedding_dim,
                 hidden_dim, 
                 vocab_size, 
                 device, pad_idx, eos_idx, sos_idx).to(device)
optimizer = optim.Adam([param for param in model.parameters() if param.requires_grad == True], lr=1.0e-3)
criterion = nn.CrossEntropyLoss(ignore_index = pad_idx)

N_EPOCHS = 20
train_losses = []

In [23]:
# train_loss = train(model, train_loader, criterion, optimizer, epoch)

In [24]:
iterator = train_loader
epoch_number = 1

In [25]:
model.train()

losses_list = []
epoch_loss = 0
progress_bar = tqdm(enumerate(iterator), total=len(iterator), desc='Epoch {}'.format(epoch_number + 1))
for idx, batch in progress_bar:
    optimizer.zero_grad()
    input_seq = torch.tensor(batch[0]).to(device)
    target_tokens = torch.tensor(batch[1]).to(device)

    break
    
    output = model(input_seq, target_tokens)
    output = output.view(-1, output.size(-1))
    target_tokens = target_tokens.view(-1)
    
    loss = criterion(output, target_tokens)
    losses_list.append(loss.item())
    progress_bar.set_postfix(train_loss = np.mean(losses_list[-100:]))
    loss.backward()

    optimizer.step()

    epoch_loss += loss.item()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


In [26]:
for x, y in iterator:
    break

In [27]:
x

tensor([[    2,  3297,  1394,  ...,  8292,  1300,  9914],
        [    2,  3297,  1394,  ...,  8292,  1300,  9914],
        [    2,  2570,  4376,  ...,     0,     0,     0],
        ...,
        [    2,  3917, 10861,  ...,  3711,  1487,  9139],
        [    2,  3308,  1585,  ...,     0,     0,     0],
        [    2,  2733,  1462,  ...,  2102,  2904,  1287]])

In [28]:
y

tensor([[   2, 1800, 1503,  ...,    0,    0,    0],
        [   2, 1411, 2461,  ...,    0,    0,    0],
        [   2, 1800, 1503,  ...,    0,    0,    0],
        ...,
        [   2, 1615, 2459,  ...,    0,    0,    0],
        [   2, 1411, 4162,  ...,    0,    0,    0],
        [   2, 1723, 5054,  ...,    0,    0,    0]])

In [32]:
for input_sequence, output_sequence in iterator:
    input_sequence = input_sequence.to(device)
    output_sequence = output_sequence.to(device)
    break

In [35]:
input_sequence.shape, output_sequence.shape

(torch.Size([64, 80]), torch.Size([64, 20]))

In [33]:
encoder_output, encoder_hidden = model.encoder(input_sequence)

In [34]:
encoder_hidden.shape

torch.Size([1, 64, 300])

In [86]:
class SimpleSequence2Sequence(torch.nn.Module):
    
    def __init__(self, vocab_size=16000, embedding_size=256,
                 hidden_size=256, num_layers=2, dropout=0.3, padding_idx=0):
        
        super().__init__()
        
        self.embedding_layer = torch.nn.Embedding(vocab_size, embedding_size, padding_idx)
        
        self.encoder = torch.nn.GRU(embedding_size, hidden_size, 
                                    num_layers=num_layers, dropout=dropout, batch_first=True)
        
        self.decoder = torch.nn.GRU(hidden_size, hidden_size, 
                                    num_layers=num_layers, dropout=dropout, batch_first=True)
        
        self.head = torch.nn.Linear(hidden_size, vocab_size)
        
    def forward(self, encoder_sequence, decoder_sequence):
        
        encoder_embed = self.embedding_layer(encoder_sequence)
        decoder_embed = self.embedding_layer(decoder_sequence)
        
        encoder_hidden, encoder_mem = self.encoder(encoder_embed)
        
        decoder_embed = torch.cat([encoder_hidden[:, -1, :].unsqueeze(1), decoder_embed], dim=1)
        
        decoder_hidden, _ = self.decoder(decoder_embed, encoder_mem)
        
        prediction = self.head(decoder_hidden)

        return prediction

In [87]:
model = SimpleSequence2Sequence()

In [88]:
model

SimpleSequence2Sequence(
  (embedding_layer): Embedding(16000, 256, padding_idx=0)
  (encoder): GRU(256, 256, num_layers=2, batch_first=True, dropout=0.3)
  (decoder): GRU(256, 256, num_layers=2, batch_first=True, dropout=0.3)
  (head): Linear(in_features=256, out_features=16000, bias=True)
)

In [89]:
for encoder_sequence, decoder_sequence in iterator:
    break

In [90]:
prediction = model(encoder_sequence, decoder_sequence)

In [91]:
prediction.shape

torch.Size([64, 21, 16000])

In [ ]:


batch_size = len(input_sequence)
outputs = torch.zeros(batch_size, 20, self.vocab_size).to(self.device)

for batch_element_index in range(batch_size):

    target_tensor = output_sequence[batch_element_index, :]

    decoder_input = torch.tensor([[self.sos_idx]], device=device)

    decoder_hidden = encoder_hidden[:,batch_element_index,:]

    encoder_output_in_current_index = encoder_output[batch_element_index,:,:]

    for di in range(20):

        decoder_output, decoder_hidden = self.decoder(
            decoder_input, decoder_hidden, encoder_output_in_current_index)

        outputs[batch_element_index,di,:] = decoder_output

        if target_tensor[di] != self.pad_idx:
            decoder_input = torch.tensor([target_tensor[di]], device=device).unsqueeze(1)
        else:
            break
return outputs

In [ ]:
pad_idx = tokenizer.vocab().index("<PAD>")
eos_idx = tokenizer.vocab().index("<EOS>")
sos_idx = tokenizer.vocab().index("<BOS>")
# Size of embedding_dim should match the dim of pre-trained word embeddings!
embedding_dim = 300
hidden_dim = 300
vocab_size = len(tokenizer.vocab())
model = My_seq2seq_attention(embedding_dim,
                 hidden_dim, 
                 vocab_size, 
                 device, pad_idx, eos_idx, sos_idx).to(device)
optimizer = optim.Adam([param for param in model.parameters() if param.requires_grad == True], lr=1.0e-3)
criterion = nn.CrossEntropyLoss(ignore_index = pad_idx)

N_EPOCHS = 20
train_losses = []
for epoch in range(N_EPOCHS):
    train_loss = train(model, train_loader, criterion, optimizer, epoch)
#     print (train_loss)
    train_losses.append(train_loss)
    if min(train_losses) == train_loss and len(train_losses) > 1:
        torch.save(model.state_dict, "best_seq2seq_attention")
        torch.save(optimizer.state_dict, "best_Adam_state_dict_attention")
    
    torch.save(model.state_dict, "last_seq2seq_attention")
    torch.save(optimizer.state_dict, "Adam_state_dict_attention")
    
    #early stopping
    validation_losses = []
    test_loss = evaluate(model, validation_loader)
    validation_losses.append(test_loss)
    
    if len(validation_losses) > 1 and validation_losses[epoch] > validation_losses[epoch-1]:
        print("stop")
        break
#     break

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
